In [0]:
import torch

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd drive/My Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive/MSRP


In [0]:
cd MSRP

[Errno 2] No such file or directory: 'MSRP'
/content/drive/My Drive/MSRP


In [0]:
from data import Data
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import f1_score
import sklearn

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
use_cuda = torch.cuda.is_available()

In [0]:
data_name = "msrp"
data_file = "./dataset/train.tsv"
training_ratio = 0.8
max_len = 20
tracking_pair = False
hidden_size = 50
batch_size = 16
num_iters = 8
learning_rate = 0.001

In [0]:
data = Data(data_name,data_file,training_ratio,max_len)

Number of Training Samples   : 3260
Number of Validation Samples   : 816


In [0]:
len(data.word2index)

12998

In [0]:
print('Number of training samples        :', len(data.x_train))
print('Number of validation samples      :', len(data.x_val))
print('Maximum sequence length           :', max_len)

Number of training samples        : 3260
Number of validation samples      : 816
Maximum sequence length           : 20


In [0]:
embd_file = "./glove-global-vectors-for-word-representation/glove.6B.100d.txt"

In [0]:
from embedding_helper import Get_Embedding

In [0]:
embedding = Get_Embedding(embd_file, data.word2index)
embedding_size = embedding.embedding_matrix.shape[1]

In [0]:
embedding_size

100

In [0]:
len(embedding.embedding_matrix)

12999

In [0]:
import torch.nn as nn
from torch import Tensor
from torch import optim
import torch.nn.functional as F

In [0]:
stops = set(stopwords.words('english'))
stopNos = []
for i in range(len(data.index2word)):
    if data.index2word[i] in stops:
        stopNos.append(i)
        
stopNos
print(len(stopNos))

0


In [0]:
def commonWords(sen_1, sen_2):
    listPairs = []
    list1 = []
    list2 = []
    for i in range(len(sen_1)):
        for j in range(len(sen_2)):
            #if sen_1[i] in stopNos:
                #continue
            
            if sen_1[i] != 0:
                if sen_1[i] == sen_2[j]:
                    list1.append(i)
                    list2.append(j)
    
    list1 = list(dict.fromkeys(list1))
    list2 = list(dict.fromkeys(list2))
    listPairs.append(list1)
    listPairs.append(list2)
    return listPairs

In [0]:
def max_pool(e_list):
  e_list = np.array(e_list)
  
  for i in range(len(e_list)):
    e_list[i] = e_list[i].data.cpu().numpy()
  mp = []
  for i in range(100):
    m = e_list[0][i]
    for j in range(len(e_list)):
      m = max(m, e_list[j][i])
    mp.append(m)
      
  #print("Length of mp = " + str(len(mp)))
  return torch.cuda.FloatTensor(mp)

In [0]:
class Manhattan_LSTM(nn.Module):
    def __init__(self, hidden_size, embedding, train_embedding = False):
        super(Manhattan_LSTM, self).__init__()
        self.use_cuda = torch.cuda.is_available()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(embedding.shape[0], embedding.shape[1])
        self.embedding.weight = nn.Parameter(embedding)
        self.input_size = embedding.shape[1]
        
        self.embedding.weight.requires_grad = train_embedding
        
        self.lstm_1 = nn.LSTM(self.input_size, self.hidden_size, num_layers=1, bidirectional=True)
        self.lstm_2 = nn.LSTM(self.input_size, self.hidden_size, num_layers=1, bidirectional=True)
        
    def exponent_neg_manhattan_distance(self, x1, x2):
        return torch.exp(-torch.sum(torch.abs(x1 - x2), dim=1))
    
    def forward(self, input, hidden):
        
        #print(input[0])
        #print(input[1])
        
        '''ip0 = input[0].t()
        ip1 = input[1].t()
        
        commonList = []
        
        for i in range(batch_size):
            listPairs = commonWords(ip0[i], ip1[i])
            commonList.append(listPairs)
    
        commonList = np.array(commonList)'''
        
        
        
        #print(commonList)
        embedded_1 = self.embedding(input[0])
        embedded_2 = self.embedding(input[1])
        
        bs = embedded_1.size()[1]
        outputs_1, hidden_1 = self.lstm_1(embedded_1, hidden)
        outputs_2, hidden_2 = self.lstm_1(embedded_2, hidden)
        
        max_pool_1 = F.adaptive_max_pool1d(outputs_1.permute(1,2,0),1).view(batch_size,-1)
        max_pool_2 = F.adaptive_max_pool1d(outputs_2.permute(1,2,0),1).view(batch_size,-1)
        #print(outputs_1.shape)
        
        #e1 = torch.tensor([])
        '''ehs_1 = []
        for i in range(batch_size):
            e_list = []
            for j in range(len(commonList[i][0])):
                x = commonList[i][0][j]
                #print(outputs_1[x][i].shape)
              
                e_list.append(outputs_1[x][i])
            if len(e_list) > 0:
                mp1 = max_pool(e_list)
            else:
                mp1 = torch.zeros(100)
              
            #torch.cat((e1,mp1), 0, out = e1)
            ehs_1.append(mp1.cuda())
        
        #print(len(e1))
        #ehs_1 = np.array(ehs_1)
        #print(ehs_1.shape)
        
        ehs_2 = []
        #e2 = torch.tensor([])
        for i in range(batch_size):
            e_list = []
            #print(commonList[i][1])
            for j in range(len(commonList[i][1])):
                x = commonList[i][1][j]
                #print(outputs_1[x][i].shape)
              
                e_list.append(outputs_2[x][i])
            if len(e_list) > 0:
                mp2 = max_pool(e_list)
            else:
                mp2 = torch.zeros(100)
              
            #torch.cat((e2,mp2), 0, out = e2)
            ehs_2.append(mp2.cuda())
            
        #print(len(e2))
        
        #print(type(ehs_2[0]))
        
        ths_1 = torch.zeros(batch_size, 200)
        for i in range(batch_size):
            ths_1[i] = torch.cat((max_pool_1[i], ehs_1[i]),0)
          
        ths_2 = torch.zeros(batch_size, 200)
        for i in range(batch_size):
            ths_2[i] = torch.cat((max_pool_2[i], ehs_2[i]),0)
          
          
        #print(ths_1)
        #print(ths_2)
        
        #print(ths_2.requires_grad)
        '''
        #similarity_scores = self.exponent_neg_manhattan_distance(ths_1.cuda(), ths_2.cuda())
        similarity_scores = self.exponent_neg_manhattan_distance(max_pool_1, max_pool_2)
        
        return similarity_scores
    
    def init_weights(self):
        for name_1, param_1 in self.lstm_1.named_parameters():
            if 'bias' in name_1:
                nn.init.constant_(param_1, 0.0)
            elif 'weight' in name_1:
                nn.init.xavier_normal_(param_1)

        lstm_1 = self.lstm_1.state_dict()
        lstm_2 = self.lstm_2.state_dict()

        for name_1, param_1 in lstm_1.items():
            # Backwards compatibility for serialized parameters.
            if isinstance(param_1, torch.nn.Parameter):
                param_1 = param_1.data

            lstm_2[name_1].copy_(param_1)

    def init_hidden(self, batch_size):
        # Hidden dimensionality : 2 (h_0, c_0) x Num. Layers * Num. Directions x Batch Size x Hidden Size
        result = torch.zeros(2, 2, batch_size, self.hidden_size)
        result = tuple(result)

        if self.use_cuda: 
            result = (result[0].cuda(), result[1].cuda())
            return result
        else: return result

In [0]:
model = Manhattan_LSTM(hidden_size, embedding.embedding_matrix, train_embedding=False)
if use_cuda: model = model.cuda()
model.init_weights()

In [0]:
import time
import random
from torch import optim
import torch.nn.utils.rnn as rnn

In [78]:
x_train = data.x_train
x_val = data.x_val
y_train = data.y_train
y_val = data.y_val
train_samples = len(x_train)
val_samples = len(x_val)
train_samples

3260

In [79]:
input_variables = x_train[0:batch_size]
print(x_train[0])

sequences_1 = [sequence[0] for sequence in input_variables]
sequences_2 = [sequence[1] for sequence in input_variables]
batch_size = len(sequences_1)

temp = rnn.pad_sequence(sequences_1 + sequences_2)
sequences_1 = temp[:, :batch_size]
sequences_2 = temp[:, batch_size:]

ip1 = sequences_1.t()
ip2 = sequences_2.t()

commonList = []

for i in range(batch_size):
    listPairs = commonWords(ip1[i], ip2[i])
    commonList.append(listPairs)
    
commonList = np.array(commonList)
print(commonList)

[tensor([1897, 3044, 6678,  124, 3032, 2149, 1322, 5811,  138,  360,  799, 1236,
         890,  800, 4656], device='cuda:0'), tensor([  980,  6649,   352,  3044,  6678,   124,  3032,  2149,   799,  1322,
         5811,   138,   360,  1444, 12241,   890,   800,  4656],
       device='cuda:0')]
[[list([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14])
  list([3, 4, 5, 6, 7, 9, 10, 11, 12, 8, 15, 16, 17])]
 [list([0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16])
  list([0, 1, 2, 3, 4, 5, 6, 8, 17, 9, 11, 12, 13, 14, 15])]
 [list([0, 1, 2, 9, 10, 11]) list([2, 3, 4, 6, 7, 8])]
 [list([0, 2, 8, 10]) list([0, 1, 4, 7])]
 [list([0, 1, 2, 3, 6, 8, 10]) list([0, 1, 2, 3, 9, 14, 4])]
 [list([2, 3, 4, 5, 7, 9, 11, 12, 13])
  list([0, 1, 6, 4, 5, 7, 9, 11, 12, 13])]
 [list([0, 1, 2, 3, 6]) list([0, 1, 2, 3, 7])]
 [list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  list([2, 3, 4, 5, 6, 7, 8, 10, 11, 12])]
 [list([0, 1, 2, 3, 4, 5]) list([0, 1, 2, 3, 4, 5])]
 [list([4, 5, 6, 7, 8, 9, 10, 11]) list([0, 1, 2, 3, 4, 

In [0]:
criterion = nn.BCELoss()
print_every = 1
print_loss_total = 0.0
train_loss = 0.0
val_loss = 0.0

In [0]:
model_trainable_parameters = tuple(filter(lambda p: p.requires_grad, model.parameters()))
model_optimizer = optim.Adam(model_trainable_parameters, lr=learning_rate)

In [82]:
hidden = model.init_hidden(batch_size)
len(hidden[0][0][0])

50

In [0]:
from helper import Helper
help_fn = Helper()

In [84]:
start = time.time()
print('Beginning Model Training.\n')

for epoch in range(0, num_iters):
    for i in range(0, train_samples, batch_size):
        input_variables = x_train[i:i+batch_size]
        similarity_scores = y_train[i:i+batch_size]
        
        sequences_1 = [sequence[0] for sequence in input_variables]
        sequences_2 = [sequence[1] for sequence in input_variables]
        batch_size = len(sequences_1)

        temp = rnn.pad_sequence(sequences_1 + sequences_2)
        sequences_1 = temp[:, :batch_size]
        sequences_2 = temp[:, batch_size:]

        if model_optimizer: model_optimizer.zero_grad()
        loss = 0.0

        hidden = model.init_hidden(batch_size)
        output_scores = model([sequences_1, sequences_2], hidden).view(-1)

        loss += criterion(output_scores, similarity_scores)
        
        loss.backward()
        model_optimizer.step()
        
        train_loss = loss
        print_loss_total += loss
     
    
    if epoch % 5:
        learning_rate *= 0.8
        model_optimizer = optim.Adam(model_trainable_parameters, lr=learning_rate)
        
    for i in range(0, val_samples, batch_size):
        input_variables = x_val[i:i+batch_size]
        actual_scores = y_val[i:i+batch_size]

        sequences_1 = [sequence[0] for sequence in input_variables]
        sequences_2 = [sequence[1] for sequence in input_variables]
        batch_size = len(sequences_1)

        temp = rnn.pad_sequence(sequences_1 + sequences_2)
        sequences_1 = temp[:, :batch_size]
        sequences_2 = temp[:, batch_size:]

        loss = 0.0
        

        hidden = model.init_hidden(batch_size)
        output_scores = model([sequences_1, sequences_2], hidden).view(-1)
        
        loss += criterion(output_scores, actual_scores)
        
        val_loss = loss
    
    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d) %.4f' % (help_fn.time_slice(start, (epoch+1) / num_iters), epoch, print_loss_avg))
        print("Training loss    " + str(train_loss.data.cpu().numpy()) + "    Validation loss:    " + str(val_loss.data.cpu().numpy()))


Beginning Model Training.

0m 3s (- 0m 25s) (0) 197.2602
Training loss    0.7620309    Validation loss:    0.7319023
0m 8s (- 0m 25s) (1) 161.5819
Training loss    0.7033289    Validation loss:    0.6310315
0m 15s (- 0m 25s) (2) 214.9561
Training loss    0.74344563    Validation loss:    0.6363233
0m 27s (- 0m 27s) (3) 404.3167
Training loss    0.70753    Validation loss:    0.66009545
0m 39s (- 0m 23s) (4) 373.2211
Training loss    0.68007183    Validation loss:    0.6703461
0m 51s (- 0m 17s) (5) 351.5277
Training loss    0.6738338    Validation loss:    0.6615599
1m 3s (- 0m 9s) (6) 338.7274
Training loss    0.66141355    Validation loss:    0.65142095
1m 16s (- 0m 0s) (7) 326.0661
Training loss    0.65344787    Validation loss:    0.6547804


In [85]:
sum_diff = 0.0
a_scores = []
p_scores = []
corr = 0
for i in range(0, val_samples, batch_size):
    input_variables = x_val[i:i+batch_size]
    actual_scores = y_val[i:i+batch_size]
    
    sequences_1 = [sequence[0] for sequence in input_variables]
    sequences_2 = [sequence[1] for sequence in input_variables]
    batch_size = len(sequences_1)

    temp = rnn.pad_sequence(sequences_1 + sequences_2)
    sequences_1 = temp[:, :batch_size]
    sequences_2 = temp[:, batch_size:]

    if model_optimizer: model_optimizer.zero_grad()
    loss = 0.0

    hidden = model.init_hidden(batch_size)
    output_scores = model([sequences_1, sequences_2], hidden).view(-1)

    #loss += criterion(output_scores, similarity_scores)
    
    
    
    for j in range(0, batch_size):
        acts = actual_scores[j].data.cpu().numpy()
        preds = output_scores[j].data.cpu().numpy()
        a_scores.append(acts)
        
        p = 0
        if preds >= 0.5:
          p = 1
        print("Actual score:    " + str(acts) + "    Predicted score:    " + str(preds))
        
        if acts == p:
          corr = corr + 1
        
        p_scores.append(p)
        sum_diff+=abs(acts-preds)
        
    

print(len(a_scores))
print(f1_score(p_scores,a_scores))
print(corr/len(a_scores))
#print(spearmanr(p_scores,a_scores))
#print(sklearn.metrics.mean_squared_error(p_scores, a_scores))

Actual score:    0.0    Predicted score:    0.2801386
Actual score:    1.0    Predicted score:    0.9317175
Actual score:    1.0    Predicted score:    0.97666395
Actual score:    0.0    Predicted score:    0.68630993
Actual score:    1.0    Predicted score:    0.6212454
Actual score:    1.0    Predicted score:    0.8868557
Actual score:    1.0    Predicted score:    0.620974
Actual score:    1.0    Predicted score:    0.40681973
Actual score:    1.0    Predicted score:    0.37990233
Actual score:    0.0    Predicted score:    0.6945236
Actual score:    0.0    Predicted score:    0.4020863
Actual score:    1.0    Predicted score:    0.80952233
Actual score:    1.0    Predicted score:    0.72558844
Actual score:    1.0    Predicted score:    0.61024314
Actual score:    1.0    Predicted score:    0.658989
Actual score:    1.0    Predicted score:    0.8917428
Actual score:    1.0    Predicted score:    0.8059238
Actual score:    0.0    Predicted score:    0.43031102
Actual score:    1.0  

In [0]:
 '''
 0.8188034188034188
 0.7401960784313726
 '''